In [1]:
import matplotlib
import random

from collections import deque, namedtuple
from itertools import count
from time import time

import gymnasium as gym
from gymnasium.vector import AsyncVectorEnv
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.utils as utils
import torch.optim as optim
import pandas as pd

is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display
# torch.autograd.set_detect_anomaly(True)

In [2]:
import sys
sys.path.append("/Users/toshi_pro/Documents/github-sub/machine-learning")
# sys.path.append("/Users/toshi/Documents/school/machine-learning")
# sys.path.append(r"C:\Users\takat\PycharmProjects\machine-learning")
import flowdata
import flowenv

raw_data_train, raw_data_test = flowdata.flow_data.using_data()
raw_data_train.dropna(how="any")
raw_data_test.dropna(how="any")

train_envs = gym.make_vec("flowenv/Flow-v1", data=raw_data_train, num_envs=4, vectorization_mode="async")
test_envs = gym.make_vec("flowenv/Flow-v1", data=raw_data_test, num_envs=4, vectorization_mode="async")


In [9]:
initial_states, _ = train_envs.reset()

In [8]:
print(len(initial_states))

4


In [18]:
class PolicyNetwork(nn.Module):
    def __init__(self, n_inputs, n_outputs):
        super(PolicyNetwork, self).__init__()
        self.common_fc = nn.Sequential(
            nn.Linear(n_inputs, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU()
        )
        self.probs = nn.Sequential(
            nn.Linear(128, n_outputs),
            nn.Softmax(dim=-1)
        )
        self.fc_value = nn.Linear(128, 1)

    def forward(self, x):
        x = self.common_fc(x)
        probs = self.probs(x)
        value = self.fc_value(x)
        return probs, value

n_input_shape = 70
n_output_shape = 2
policy_net = PolicyNetwork(n_input_shape, n_output_shape)
optimizer = optim.Adam(policy_net.parameters(), lr=1e-5)


In [19]:
confusion_matrix = np.zeros((2,2), dtype=int)
one_episode_rewards = []
one_episode_log_probs = []
states = torch.tensor(initial_states, dtype=torch.float32)
prob_distribution, value = policy_net(states)

actions = torch.multinomial(prob_distribution, num_samples=1)
print(actions)

tensor([[1],
        [0],
        [1],
        [0]])


In [20]:
next_states, rewards, terminated, truncated, info = train_envs.step(actions)

In [21]:
print(info["confusion_position"])
for item in info["confusion_position"]:
    confusion_matrix[item[0], item[1]] += 1

rewards = torch.tensor(rewards, dtype=torch.float32)
one_episode_rewards.append(rewards)
log_probs = prob_distribution.gather(1, actions)
one_episode_log_probs.append(torch.log(log_probs))

[(1, 0) (1, 1) (1, 0) (1, 1)]


In [23]:
print(True in terminated)
if True in terminated:
    next_states 

False
